In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import cvlib as cv

In [6]:
from cvlib.object_detection import draw_bbox

In [9]:
from numpy.polynomial import polynomial

In [ ]:
image = cv.imread("Cars.jpg")

In [ ]:
plt.imshow(image)